In [1]:
# THis version is based on https://www.youtube.com/watch?v=0CBNLEP9Vyc,
# Since some commands are not available, some update is made by yanjun zhang
# All update is commented  2024-03-04

In [2]:
# 
from dolfinx import default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, Function, FunctionSpace, assemble_scalar,
                         form, locate_dofs_geometrical, locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import create_unit_square, locate_entities
from dolfinx.plot import vtk_mesh

from ufl import (SpatialCoordinate, TestFunction, TrialFunction, dx, grad, inner)
from mpi4py import MPI
import meshio
import gmsh

import ufl
from petsc4py import PETSc


from dolfinx import *
from dolfinx import fem, mesh, plot, io 
import dolfinx
import numpy as np

In [3]:
# Define temporal parameters

t = 0       # start time
T = 20.0    # end time
n_steps = 50
dt = T/n_steps    # time step

# coefficient of friction
k = 100      # thermal conductivity
u_in = 20    # inlet temperature
u_out = -20  # outlet temperature


In [4]:
# Define mesh
# THis work required previous step in: import mesh.ipynb, which converts the msh file to mesh.xdmf and mt.xdmf

with XDMFFile(MPI.COMM_WORLD, "mesh.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="Grid")
    cd = xdmf.read_meshtags(mesh, name="Grid")    # volume mesh
    
mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim - 1)


with XDMFFile(MPI.COMM_WORLD, "mt.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="Grid")
    fd = xdmf.read_meshtags(mesh, name="Grid")    # face element


In [5]:
# Define function space
# Degrees of freedom on the notes, first order finite element in lagrange polyniominal

V = fem.FunctionSpace(mesh, ("Lagrange", 1))


In [6]:
# volume cell check
volume_cells = cd.find(16)
print('Volume cell number is','\n', volume_cells)

Volume cell number is 
 [    0     1     2 ... 26645 26646 26647]


In [17]:
# Define boundary and initial conditions

# bottom_cells = ct.find(bottom_narker),  
# bottom_marker is 2 or 3 whaich is defined in gmsh
# these information from tutorials: Defining subdomains for different materials

inlet_cells = fd.find(2)
out_cells = fd.find(3)

inlet_dofs = locate_dofs_topological(V, mesh.topology.dim - 1, inlet_cells)

# here 20 is inlet temperature, dokken has suggest this, source is below:
# https://fenicsproject.discourse.group/t/import-gmsh-to-fenicsx-then-define-two-dirichlet-boundary-condition/13823/2
bc1 = dirichletbc(default_scalar_type(20), inlet_dofs, V)


# https://jsdokken.com/dolfinx-tutorial/chapter3/component_bc.html
out_dofs = locate_dofs_topological(V, mesh.topology.dim - 1, out_cells)
bc2 = dirichletbc(default_scalar_type(-20), out_dofs, V)

bc = [bc1, bc2]


In [18]:
print('1: Inlet_dofs is', '\n', inlet_dofs)
print('2: Out_dofs is', '\n', out_dofs)
print('3: Bc is', '\n', bc[0], bc[1])
# print('3: Inlet = np.full_like = ', '\n', inlet)

1: Inlet_dofs is 
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 33 34 35 37 38 39 41 45 46]
2: Out_dofs is 
 [4078 4079 4080 4081 4082 4083 4084 4086 4087 4089 4090 4091 4092 4093
 4094 4095 4096 4098 4099 4100 4101 4103 4104 4105 4106 4107 4108 4109
 4110 4111 4112 4113 4114 4115 4116 4117 4119 4120 4121 4122 4123 4125
 4126 4127 4130 4131 4132 4133]
3: Bc is 
 <dolfinx.fem.bcs.DirichletBC object at 0x7f5cef181600> <dolfinx.fem.bcs.DirichletBC object at 0x7f5cef1815a0>


In [29]:
# no use temporary
def initial_condition(x):
    return 20

# Time-dependent output

In [30]:
xdmf = io.XDMFFile(mesh.comm, "diffusion.xdmf", "w")
xdmf.write_mesh(mesh)

# Define solution variable, and interpolate initial solution for visualization in Paraview
uh = fem.Function(V)
uh.name = "uh"

#uh.interpolate(initial_condition)
xdmf.write_function(uh, t)

HDF5-DIAG: Error detected in HDF5 (1.10.7) MPI-process 0:
  #000: ../../../src/H5F.c line 366 in H5Fcreate(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #001: ../../../src/H5Fint.c line 1677 in H5F_open(): unable to truncate a file which is already open
    major: File accessibility
    minor: Unable to open file


RuntimeError: Failed to create HDF5 file.

# Variational problem and solver

In [22]:
u = ufl.TrialFunction(V) 
v = ufl.TestFunction(V)
f = fem.Constant(mesh, PETSc.ScalarType(0))
u_n = Function(V)

a = u * v * ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx 
L = (u_n + dt * f) * v * ufl.dx


In [23]:
print(v)
print(f)
print(u_n)
print(a)
print(L)

v_0
c_1
f
{ v_0 * v_1 } * dx(<Mesh #1>[everywhere], {})
  +  { 0.4 * ((grad(v_1)) . (grad(v_0))) } * dx(<Mesh #1>[everywhere], {})
{ v_0 * (f + 0.4 * c_1) } * dx(<Mesh #1>[everywhere], {})


# Preparing linear algerba structures for time dependent problems

In [24]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)
print(linear_form)

In [25]:
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc
#from petsc4py import PETSc

A = assemble_matrix(bilinear_form, bc)
A.assemble()
#print(A.view())
b = create_vector(linear_form)
print(b.view())

None
Vec Object: 1 MPI processes
  type: mpi
Process [0]
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0.
0

# Using petsc4py to create a linear solver

In [26]:
solver = PETSc.KSP().create(mesh.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)


# Visualization of time dependent problem using pyvista

In [28]:
import matplotlib as mpl
import pyvista
pyvista.start_xvfb()

grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V))

for i in range(n_steps):
    t += dt

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)

    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [bilinear_form], [bc])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, bc)

    # Solve linear problem
    solver.solve(b, uh.vector)
    uh.x.scatter_forward()

    # Update solution at previous time step (u_n)
    u_n.x.array[:] = uh.x.array

    # Write solution to file
    xdmf.write_function(uh, t)
    # Update plot
    warped = grid.warp_by_scalar("uh", factor=1)
    plotter.update_coordinates(warped.points.copy(), render=False)
    plotter.update_scalars(uh.x.array, render=False)
    plotter.write_frame()
plotter.close()
xdmf.close()

KeyError: 'Data array (uh) not present in this dataset.'